Mögliche Labels:
- eindeutige Positionierung der Zielspalte
- mehrfache Positionierungen der Zielspalten sortiert nach derer Kosinus-Ähnlichkeit

In [1]:
from utils.preprocessing import WordEmbedding, load_word_emb
import random
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.utils import shuffle

from copy import copy

import warnings
warnings.filterwarnings("ignore")

In [2]:
w2v_config = {
    'data_dir': 'data/glove',
    'word2idx_path': 'word2idx.json',
    'usedwordemb_path': 'usedwordemb.npy'
}
w2v = WordEmbedding(load_word_emb(w2v_config['data_dir'], 
                                  w2v_config['word2idx_path'],
                                  w2v_config['usedwordemb_path']))

In [3]:
# load table schemata
with open('data/wikisql/tables.jsonl') as file:
    table_schemata = pd.DataFrame([json.loads(line) for line in file.readlines()])
table_schemata['fits_1toN'] = ''
table_schemata['fits_1to0'] = ''
table_schemata['header'] = table_schemata['header'].apply(lambda x: '<|>'.join(x)) # needed to drop duplicates
table_schemata = table_schemata[['header', 'fits_1toN', 'fits_1to0']].drop_duplicates().reset_index(drop=True)
table_schemata['header'] = table_schemata['header'].apply(lambda x: x.split('<|>')) # rebuild original state
table_schemata.head()

,header,fits_1toN,fits_1to0
0,"[Date, Time, ACC Team, Big Ten Team, Location,...",,
1,"[Institution, Wins, Loss, Home Wins, Home Loss...",,
2,"[Pick #, MLS Team, Player, Position, Affiliation]",,
3,"[DVD title, Number of Episodes, Region 2, Regi...",,
4,"[Year, Coach, Crew, Record, Win %]",,


In [4]:
#table_schemata = table_schemata.head(10000)

In [5]:
all_headers = []
_ = [all_headers.extend(header) for header in table_schemata['header'].values]
all_headers = list(set(all_headers))
random.shuffle(all_headers)
candidates = [column \
              .replace('/', ' ') \
              .replace('_', ' ') \
              for column in all_headers[:5000]]
cache = {candidate: np.mean([w2v(word.lower()) for word in candidate.split()], axis=0) for candidate in candidates}
if cache.get(''):
    del cache['']

In [ ]:
def calculate_1to1(header, cache, threshold=0.8):
    eps = 0.001
    embs = [np.mean([w2v(word.lower()) for word in col.split()], axis=0) for col in header]
    fits = defaultdict(list)
    for candidate, embedding in cache.items():
        try:
            vectors = [embedding] + embs
            similarity = cosine_similarity(vectors)[0][1:]
            max_sim = np.max(similarity)
            if abs(max_sim + eps) >= 1.0 or max_sim < threshold:
                continue
            max_pos = np.argmax(similarity)

            # append the best candidate with its similarity to the header
            # to the lists of candidates for the given header
            # i.e. fits['ACC Team'] -> [('Team', 0.75), ('Coach', 0.64), (<new candidate>, <similarity>)]
            fits[header[max_pos]].append((candidate, max_sim))
        except:
            continue
    return fits    

def calculate_1t0(header, cache, threshold=0.6):
    embs = [np.mean([w2v(word.lower()) for word in col.split()], axis=0) for col in header]
    fits = defaultdict(list)
    for candidate, embedding in cache.items():
        try:
            vectors = [embedding] + embs
            similarity = cosine_similarity(vectors)[0][1:]
            max_sim = np.max(similarity)
            # if the candidate is not similar to any of the columns, append it to the fits
            if max_sim < threshold:
                min_sim = np.min(similarity)
                min_pos = np.argmin(similarity)
                fits[header[min_pos]].append((candidate, min_sim))
        except:
            continue
    return fits

def reduce_fits(good_fits, increase_similarity=True, lower_threshold=0.9, upper_threshold=0.3):
    best_fits = {}
    for column, candidate_tuples in good_fits.items():
        best_candidate = sorted(candidate_tuples, key=lambda x: x[1], reverse=True if increase_similarity else False)[0]
        similarity = best_candidate[1]
        if increase_similarity:
            if similarity > lower_threshold:
                best_fits[column] = best_candidate[0]
        else:
            if similarity < upper_threshold:
                best_fits[column] = best_candidate[0]
    return best_fits

def expand_targets(header, cache, fits_1to1, threshold=0.7):
    fits_1toN = fits_1to1.copy()
    for source_col, target_col in fits_1to1.items():
        other_cols = header[:header.index(target_col)] + header[header.index(target_col) + 1:]
        embs = [np.mean([w2v(word.lower()) for word in col.split()], axis=0) for col in other_cols]
        vectors = embs + [cache[source_col]]
        similarity = cosine_similarity(vectors)[0][1:]
        indexed_similarity = sorted(enumerate(similarity), key=lambda item: item[1], reverse=True)
        for idx, sim in indexed_similarity:
            if sim > threshold:
                fits_1toN[source_col] += '<|>{}'.format(other_cols[idx])
    return fits_1toN

for idx, row in tqdm(table_schemata.iterrows(), total=len(table_schemata)):
    header = row['header']
    fits_1to1 = calculate_1to1(header, cache)
    fits_1to1 = reduce_fits(fits_1to1, increase_similarity=True)
    
    fits_1to0 = calculate_1t0(header, cache)
    fits_1to0 = reduce_fits(fits_1to0, increase_similarity=False)
    
    # swap order from target_col:sourc_col to source_col: target_col
    # implementation originally done in the target_col:source_col format
    # as this is easier to handle for candidate reduction
    fits_1to1 = {value: key for key, value in fits_1to1.items()}
    fits_1to0 = {value: key for key, value in fits_1to0.items()}
    
    fits_1toN = expand_targets(header, cache, fits_1to1, threshold=0.75)
    
    row['fits_1toN'] = fits_1toN
    row['fits_1to0'] = fits_1to0
    table_schemata.loc[idx] = row

In [ ]:
table_schemata.head()

In [ ]:
# Drop rows that have not been tested
table_schemata.drop(table_schemata.loc[(table_schemata['fits_1toN'] == '') & \
                                       (table_schemata['fits_1to0'] == '')].index, inplace=True)

In [ ]:
rows_1toN = []
rows_1to0 = []
total_rows = []
ratio_1to0_1toN = 0.2 # relative ratio of how many 1to0 cases exist in comparison to 1toN cases
for idx, ts_row in tqdm(table_schemata.iterrows()):
    header, fits_1toN = ts_row[['header', 'fits_1toN']]
    num_words_in_header = len((' '.join(header)).split(' '))
    if num_words_in_header > 30:
        continue
    for source_col, target_cols in fits_1toN.items(): 
        if 'k {\math' in source_col:
            continue
        seq_row = {
            'source_col': source_col,
            'input_cols': '<|>'.join(header),
            'target_cols': '<|>'.join(target_cols.split('<|>')) # first create a list from the joined 
                        # target cols, then concatenate them (needs to be done to serialize them correctly)
        }
        rows_1toN.append(seq_row)
    for source_col, _ in fits_1to0.items(): 
        if 'k {\math' in source_col:
            continue
        seq_row = {
            'source_col': source_col,
            'input_cols': '<|>'.join(header),
            'target_cols': '<NONE>'
        }
        rows_1to0.append(seq_row)
amount_1to0 = int(len(rows_1toN) * ratio_1to0_1toN)
rows_1toN, rows_1to0 = shuffle(rows_1toN), shuffle(rows_1to0)[:amount_1to0]

total_rows = shuffle(rows_1toN + rows_1to0)

total_rows = pd.DataFrame(total_rows)
total_rows.to_csv('data/training/schema_matching_raw_1toN.txt', index=False)

In [ ]:
total_rows.head()

In [ ]:
num_1to0 = len(total_rows[total_rows['target_cols'] == '<NONE>'])
num_1toN = len(total_rows[(total_rows['target_cols'].str.contains('<|>')) &
                          (total_rows['target_cols'] != '<NONE>')])
num_1to1 = len(total_rows) - num_1toN - num_1to0
num_1to0, num_1to1, num_1toN

In [ ]:
total_rows.shape

Vorgehen:
1. für jede Tabelle eine Spalte finden, die aus Quellschema sein könnte -> die als Eingabe  
Format: Quellspalte -> [Zielspalte]